# Quiz
> tmbd_5000_movies에서 overview를 이용한 추천 시스템을 구축하시오
* 코드 완성후 [Quiz_조현기.ipynb]로 제출

In [34]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#1. 데이터 읽어오기 및 사용할 컬럼 추리기

In [35]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Lec4_BigData/5.컨텐츠기반/data/tmdb_5000_movies.csv')
df.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

> 고객이 영화를 평가할 수 있는 정보를 가지고 있는 컬럼만 잘라보도록 한다.

In [37]:
df_movies = df[['title','budget', 'genres', 'keywords', 'overview', 'popularity','release_date', 'runtime', 'vote_average', 'vote_count']]
df_movies.head()

,title,budget,genres,keywords,overview,popularity,release_date,runtime,vote_average,vote_count
0,Avatar,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",150.437577,2009-12-10,162.0,7.2,11800
1,Pirates of the Caribbean: At World's End,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",139.082615,2007-05-19,169.0,6.9,4500
2,Spectre,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,107.376788,2015-10-26,148.0,6.3,4466
3,The Dark Knight Rises,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,112.312950,2012-07-16,165.0,7.6,9106
4,John Carter,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...",43.926995,2012-03-07,132.0,6.1,2124


#2. 결측치 확인

In [38]:
df_movies.isna().sum()

title           0
budget          0
genres          0
keywords        0
overview        3
popularity      0
release_date    1
runtime         2
vote_average    0
vote_count      0
dtype: int64

In [39]:
df_movies[df_movies['overview'].isna()]

,title,budget,genres,keywords,overview,popularity,release_date,runtime,vote_average,vote_count
2656,Chiamatemi Francesco - Il Papa della gente,15000000,"[{""id"": 18, ""name"": ""Drama""}]","[{""id"": 717, ""name"": ""pope""}, {""id"": 5565, ""na...",NaN,0.738646,2015-12-03,NaN,7.3,12
4140,"To Be Frank, Sinatra at 100",2,"[{""id"": 99, ""name"": ""Documentary""}]","[{""id"": 6027, ""name"": ""music""}, {""id"": 225822,...",NaN,0.050625,2015-12-12,NaN,0.0,0
4431,Food Chains,913000,"[{""id"": 99, ""name"": ""Documentary""}]",[],NaN,0.795698,2014-04-26,83.0,7.4,8


> Overview로 영화 추천을 해야하기 때문에 결측치를 처리해줘야 한다. 이 세 영화는 vote_average가 낮고 vote_count도 최대 12 정도라 채워준다고 해도 추천되지는 않을것 같다. 따라서 drop해도 될 것 같다.

>그러나 채워봤다.

In [40]:
df_movies['overview'].iloc[2656] = 'The film tells the story of the life of Pope Francis (formerly Jorge Mario Bergoglio), from his youth in Buenos Aires and his experiences under the dictatorship of Jorge Rafael Videla to his appointment as pope in 2013.'
df_movies['runtime'].iloc[2656] = 113.0
df_movies['runtime'].iloc[2656]

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


113.0

In [41]:
df_movies['overview'].iloc[4140] = 'The life of Frank Sinatra, as an actor and singer and the steps along the way that led him to become such an icon.'
df_movies['runtime'].iloc[4140] = 81.0
df_movies['runtime'].iloc[4140]

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


81.0

In [42]:
df_movies['overview'].iloc[4431] = 'Thought-provoking docu stresses economics of Big Grocery.'
df_movies['overview'].iloc[4431]

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


'Thought-provoking docu stresses economics of Big Grocery.'

> 그러나 검색해보니 찾기 쉬워 채워봤다.

In [43]:
df_movies.isna().sum()

title           0
budget          0
genres          0
keywords        0
overview        0
popularity      0
release_date    1
runtime         0
vote_average    0
vote_count      0
dtype: int64

In [44]:
df_movies[df_movies['release_date'].isna()]

,title,budget,genres,keywords,overview,popularity,release_date,runtime,vote_average,vote_count
4553,America Is Still the Place,0,[],[],1971 post civil rights San Francisco seemed li...,0.0,NaN,0.0,0.0,0


> 마지막남은 결측치인 release_data는 title과 overview외엔 아무 정보도 들어있지 않은것으로 판단된다. 또한 google에 검색해도 제대로 된 정보가 나오지 않는다.

> 이것 또한 평점이 없으니 정렬에서 탈락할 것이다.

#3. 이상치 확인

In [45]:
df_movies.describe()

,budget,popularity,runtime,vote_average,vote_count
count,4.803000e+03,4803.000000,4803.000000,4803.000000,4803.000000
mean,2.904504e+07,21.492301,106.871747,6.092172,690.217989
std,4.072239e+07,31.816650,22.610481,1.194612,1234.585891
min,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,7.900000e+05,4.668070,94.000000,5.600000,54.000000
50%,1.500000e+07,12.921594,103.000000,6.200000,235.000000
75%,4.000000e+07,28.313505,117.500000,6.800000,737.000000
max,3.800000e+08,875.581305,338.000000,10.000000,13752.000000


In [46]:
df_movies.sort_values('runtime', ascending=True).head(10)

,title,budget,genres,keywords,overview,popularity,release_date,runtime,vote_average,vote_count
4584,Love in the Time of Monsters,0,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 35, ""nam...",[],Two sisters travel to a cheesy tourist trap wh...,0.133619,2014-03-08,0.0,5.0,2
3953,Fort McCoy,0,"[{""id"": 10752, ""name"": ""War""}, {""id"": 18, ""nam...","[{""id"": 187056, ""name"": ""woman director""}]",Unable to serve in World War II because of a h...,0.384496,2014-01-01,0.0,6.3,2
4581,Gory Gory Hallelujah,0,[],"[{""id"": 187056, ""name"": ""woman director""}]",Four actors compete for the role of Jesus - a ...,0.033883,2003-11-05,0.0,1.0,1
4322,Sex With Strangers,0,"[{""id"": 99, ""name"": ""Documentary""}]",[],"For some married couples, sex is an obsession ...",0.014406,2002-02-22,0.0,5.0,1
4328,The Vatican Exorcisms,0,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 99, ""nam...","[{""id"": 2626, ""name"": ""exorcism""}]",Documentary following US film-maker Joe Marino...,0.447166,2013-05-29,0.0,4.4,11
4435,The Horror Network Vol. 1,0,"[{""id"": 27, ""name"": ""Horror""}]","[{""id"": 9706, ""name"": ""anthology""}]","Serial killers, ghostly phone calls, inner dem...",0.392658,2015-10-27,0.0,5.0,2
4241,The Salon,1500000,"[{""id"": 35, ""name"": ""Comedy""}]","[{""id"": 10183, ""name"": ""independent film""}]",A Beauty shop owner finds romance as she strug...,2.028170,2005-01-13,0.0,3.5,1
4502,Water & Power,0,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name...",[],"Twin brothers nicknamed ""Water"" and ""Power"" fr...",0.350557,2014-05-02,0.0,3.0,1
4313,Below Zero,0,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 27, ""n...","[{""id"": 888, ""name"": ""screenwriter""}]",When Jack (Edward Furlong) is in danger of mis...,1.365140,2011-10-22,0.0,4.4,12
4458,Harrison Montgomery,0,[],[],Film from Daniel Davila,0.006943,2008-01-01,0.0,0.0,0


> vote_average와 vote_count 이상치는 투표한 사람이 없을수도 있기때문에 이상이 아닐 가능성이 있고, 또한 평가가 없으니 추천되지 않을것이기 때문에 이상치를 채우지 않아도 될 것 같다.

> 또한 runtime의 이상치는 최대치의 영화는 검색해본 결과 실제 존재하는 영화이고, 러닝타임이 0인 영화는 정보에 이상이 있는것 같지만, 영화 추천은 runtime과 무관하므로 이상치처리는 넘어가도록 하겠다.

#4. 데이터 가공

In [47]:
df_movies.loc[1:1, 'genres'].values[0][0]

'['

> 현재 dict형식으로 입력되었을 정보가 여기서는 str로 입력되어 인덱싱하면 전체 정보가 아닌 '['만 출력된 것을 알 수 있다.

In [48]:
def StrtoList(data_frame, col):
  from ast import literal_eval
  data_frame[col] = data_frame[col].apply(literal_eval)
  data_frame[col] = data_frame[col].apply(lambda x: [y['name'] for y in x])
  data_frame[col] = data_frame[col].apply(lambda x: (' , ').join(x))
  return data_frame

df_movies = StrtoList(df_movies, 'genres')
df_movies = StrtoList(df_movies, 'keywords')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [49]:
df_movies.head(1)

,title,budget,genres,keywords,overview,popularity,release_date,runtime,vote_average,vote_count
0,Avatar,237000000,"Action , Adventure , Fantasy , Science Fiction","culture clash , future , space war , space col...","In the 22nd century, a paraplegic Marine is di...",150.437577,2009-12-10,162.0,7.2,11800


> str로 저장되어 있는 genres와 keywords를 str-> dict -> list형으로 바꿔주었다.

#5. 유사도 분석

In [50]:
def get_Similarity_Sorted_index(data_frame, col):
  from sklearn.feature_extraction.text import TfidfVectorizer
  from sklearn.metrics.pairwise import cosine_similarity

  #Tfidf로 문서빈도수를 구해본다.
  tfidf_vect = TfidfVectorizer(ngram_range=(1, 2)).fit(df_movies[col])

  overview_matrix = tfidf_vect.fit_transform(data_frame[col])

  #코사인 유사도분석을 진행한다.
  #단어의 빈도수가 아닌 문서간의 각도를 기준으로 유사도를 판별하는 알고리즘
  overview_similarity = cosine_similarity(overview_matrix, overview_matrix)

  #argsort는 기본 오름차순으로 정렬하지만, 유사도가 높은 순으로 내림차순으로 정렬한다.
  overview_similarity_sorted_index = overview_similarity.argsort()[:, ::-1]
  return overview_similarity_sorted_index

#get_Similarity_Sorted_index(df_movies, 'overview')

> 유사도 분석을 위한 함수를 만들었다.


In [51]:
overview_Simliarity_sorted_list = get_Similarity_Sorted_index(df_movies, 'overview')

#6. 영화 제목으로 검색해서 비슷한 영화 리스트 찾기

In [52]:
def findSimilarMovies(data_frame, sim_forted_list, movie_name, col='title'):
  movie_index = data_frame[data_frame[col].str.lower() == movie_name.lower()].index[0]
  similar_movie_index_list = overview_Simliarity_sorted_list[movie_index, :20]

  if movie_index in similar_movie_index_list:
    similar_movie_index_list = overview_Simliarity_sorted_list[movie_index, :21]
    similar_movie_index_list = similar_movie_index_list[similar_movie_index_list != movie_index]
  
  return similar_movie_index_list

findSimilarMovies(df_movies, overview_Simliarity_sorted_list, 'titanic')

array([ 770,  378, 2287, 4287, 4610, 4044, 2446, 2129, 2239, 2289, 1269,
       4622,  861, 2930, 3212, 1875, 3254, 2143,  296,  171])

#7. 가중치를 넣어 영화 정렬하기

> 가중치 넣기

In [53]:
#전체 영화에 대한 평균 평점
C = df_movies['vote_average'].mean()
#평점을 부여하기 위한 최소 투표 횟수
m = df_movies['vote_count'].quantile(0.6)
# 개별 영화에 평점을 투표한 횟수
v = df_movies['vote_count']
#개별 영화에 대한 평균 평점
R = df_movies['vote_average']

df_movies['weighted_vote'] = (v/(v+m)) * R + (m/(v+m)) * C
df_movies['weighted_vote']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


0       7.166301
1       6.838594
2       6.284091
3       7.541095
4       6.098838
          ...   
4798    6.290894
4799    6.089611
4800    6.106650
4801    6.084894
4802    6.100782
Name: weighted_vote, Length: 4803, dtype: float64

> 정렬한 결과 출력하기

In [54]:
def getTop10(data_frame, sim_list):

  sorted_df = data_frame.iloc[sim_list].sort_values('weighted_vote', ascending=False).iloc[0:10]
  collist = sorted_df.columns
  for i in range(10):
    print("Number", i+1, sep="")
    vallist = sorted_df.iloc[i].values

    for j in range(len(collist)-1):
      if j not in [3, 8] :
        print(collist[j], ":", vallist[j])
    print()

#7. 사용자의 입력을 받고 정렬해서 출력하기

In [55]:
#movie_name = 'titanic'

movie_name = input('Please input the movie name: ')
sim_list = findSimilarMovies(df_movies, overview_Simliarity_sorted_list, movie_name)

getTop10(df_movies, sim_list)

Please input the movie name: avatar
Number1
title : The Matrix
budget : 63000000
genres : Action , Science Fiction
overview : Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.
popularity : 104.309993
release_date : 1999-03-30
runtime : 136.0
vote_count : 8907

Number2
title : The Exorcist
budget : 8000000
genres : Drama , Horror , Thriller
overview : 12-year-old Regan MacNeil begins to adapt an explicit new personality as strange events befall the local area of Georgetown. Her mother becomes torn between science and superstition in a desperate bid to save her daughter, and ultimately turns to her last hope: Father Damien Karras, a troubled priest who is struggling with his own faith.
popularity : 47.584643
release_date : 1973-12-26
runtime : 122.0
vote_count : 2005

Number3
title : Despicable Me
budget : 69000000
genres : Animation , Family
overview : Vill

#강사님 코드

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Lec4_BigData/5.컨텐츠기반/data/tmdb_5000_movies.csv')
df.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [ ]:
df_movies = df[['title','budget', 'genres', 'keywords', 'overview', 'popularity','release_date', 'runtime', 'vote_average', 'vote_count']]
df_movies.head()

,title,budget,genres,keywords,overview,popularity,release_date,runtime,vote_average,vote_count
0,Avatar,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",150.437577,2009-12-10,162.0,7.2,11800
1,Pirates of the Caribbean: At World's End,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",139.082615,2007-05-19,169.0,6.9,4500
2,Spectre,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,107.376788,2015-10-26,148.0,6.3,4466
3,The Dark Knight Rises,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,112.312950,2012-07-16,165.0,7.6,9106
4,John Carter,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...",43.926995,2012-03-07,132.0,6.1,2124


In [ ]:
df_movies = df_movies.dropna()

##2.1 소문자 처리

In [ ]:
df_movies['overview'] = df_movies['overview'].apply(lambda x : x.lower())
df_movies['overview']

0       in the 22nd century, a paraplegic marine is di...
1       captain barbossa, long believed to be dead, ha...
2       a cryptic message from bond’s past sends him o...
3       following the death of district attorney harve...
4       john carter is a war-weary, former military ca...
                              ...                        
4798    el mariachi just wants to play his guitar and ...
4799    a newlywed couple's honeymoon is upended by th...
4800    "signed, sealed, delivered" introduces a dedic...
4801    when ambitious new york attorney sam is sent t...
4802    ever since the second grade when he first saw ...
Name: overview, Length: 4799, dtype: object

##2.2 불필요한 단어 제거

In [ ]:
word_list = [' on ', ' in ', ' the ', ' a ', ' is ', ' to ', ' but ', ' and ', ' an ', ' be ', ' has ', ' have ', ' of ', ' for ', ' with ', ' will ', ' not ', ' as ', ' it ', "'s ", ' who ', ' what ', ' which ', ' that ', ' him ', ' her ', ' i ', ' you ', ' she ', ' he ']
testwords = df_movies['overview'][3]

for word in word_list:
  testwords = testwords.replace(word, ' ')
testwords

'following death district attorney harvey dent, batman assumes responsibility dent crimes protect late attorney reputation subsequently hunted by gotham city police department. eight years later, batman encounters mysterious selina kyle villainous bane, new terrorist leader overwhelms gotham finest. dark knight resurfaces protect city branded enemy.'

In [ ]:
def replaceWord(words):
  word_list = [' on ', ' in ', ' the ', ' a ', ' is ', ' to ', ' but ', ' and ', ' an ', ' be ', ' has ', ' have ', ' of ', ' for ', ' with ', ' will ', ' not ', ' as ', ' it ', "'s ", ' who ', ' what ', ' which ', ' that ', ' him ', ' her ', ' i ', ' you ', ' she ', ' he ']
  for word in word_list:
    words = words.replace(word, ' ')
  return words

In [ ]:
df_movies['overview'] = df_movies['overview'].apply(lambda x: replaceWord(x))
df_movies['overview'].head()

0    in 22nd century, paraplegic marine dispatched ...
1    captain barbossa, long believed dead, come bac...
2    a cryptic message from bond’s past sends trail...
3    following death district attorney harvey dent,...
4    john carter war-weary, former military captain...
Name: overview, dtype: object

##2.3 데이터 count

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer()
overview_matrix = vect.fit_transform(df_movies['overview'])
print(sorted(vect.vocabulary_.items()))
overview_matrix.shape

[('00', 0), ('000', 1), ('007', 2), ('07am', 3), ('10', 4), ('100', 5), ('1000', 6), ('101', 7), ('108', 8), ('10th', 9), ('11', 10), ('114', 11), ('117', 12), ('118', 13), ('119', 14), ('11th', 15), ('12', 16), ('1200', 17), ('1215', 18), ('1250', 19), ('125th', 20), ('12th', 21), ('13', 22), ('1300', 23), ('13th', 24), ('14', 25), ('140', 26), ('1408', 27), ('142', 28), ('1429', 29), ('148', 30), ('14pm', 31), ('14th', 32), ('15', 33), ('150', 34), ('150th', 35), ('1520s', 36), ('1536', 37), ('15th', 38), ('16', 39), ('1600s', 40), ('161', 41), ('1630s', 42), ('1644', 43), ('1681', 44), ('1691', 45), ('16th', 46), ('17', 47), ('170', 48), ('1700s', 49), ('173rd', 50), ('1748', 51), ('1776', 52), ('17th', 53), ('18', 54), ('180', 55), ('1800', 56), ('1818', 57), ('1820', 58), ('1820s', 59), ('1824', 60), ('1831', 61), ('1834', 62), ('1836', 63), ('1838', 64), ('1839', 65), ('1841', 66), ('1845', 67), ('1850', 68), ('1850s', 69), ('1856', 70), ('1857', 71), ('1860', 72), ('1862', 73), 

(4799, 21261)

#3. 유사도 분석

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

overview_similarity = cosine_similarity(overview_matrix, overview_matrix)
overview_similarity[:10]

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.01760025, ..., 0.01952914, 0.        ,
        0.        ],
       [0.        , 0.01760025, 1.        , ..., 0.01771795, 0.        ,
        0.00399631],
       ...,
       [0.03552293, 0.02093294, 0.03352901, ..., 0.00320764, 0.00651512,
        0.00783234],
       [0.        , 0.        , 0.02271991, ..., 0.02700255, 0.0381086 ,
        0.02490088],
       [0.        , 0.        , 0.01028291, ..., 0.01427212, 0.00482812,
        0.02194191]])

In [ ]:
overview_similarity_sorted = overview_similarity.argsort()[:, ::-1]
overview_similarity_sorted[:3]

array([[   0, 3603, 2130, ..., 3024, 3023, 2399],
       [   1, 2542, 3094, ..., 2872, 2871,    0],
       [   2, 1343, 4070, ..., 1844, 1843,    0]])

#4. 추천

In [ ]:
C = df_movies['vote_average'].mean()
m = df_movies['vote_count'].quantile(0.6)
print('c : ', round(C, 3))
print('m : ', round(m, 3))

c :  6.094
m :  371.0


In [ ]:
def weighted_vote_average(data_frame):
 # C = data_frame['vote_average'].mean()
  #m = data_frame['vote_count'].quantile(0.6)
  v = data_frame['vote_count']
  R = data_frame['vote_average']
  return (v/(v+m))*R + (m/(v+m))*C

df_movies['weighted_vote'] = df_movies.apply(weighted_vote_average, axis=1)
df_movies[:3]

,title,budget,genres,keywords,overview,popularity,release_date,runtime,vote_average,vote_count,weighted_vote
0,Avatar,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","in 22nd century, paraplegic marine dispatched ...",150.437577,2009-12-10,162.0,7.2,11800,7.166292
1,Pirates of the Caribbean: At World's End,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","captain barbossa, long believed dead, come bac...",139.082615,2007-05-19,169.0,6.9,4500,6.838625
2,Spectre,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",a cryptic message from bond’s past sends trail...,107.376788,2015-10-26,148.0,6.3,4466,6.284214


In [ ]:
def find_movie(dataframe, overview_similarity_sorted, movie_name, top=10):
  #영화 제목으로 index 정보 얻기
  #영화 제목을 소문자화 처리해야한다.
  title_movie = dataframe[dataframe['title'].str.lower().isin([movie_name.lower()])]
  title_idx = title_movie.index.values

  #index정보를 이용하여 상위 20개의 추천영화 얻기
  movie_list = overview_similarity_sorted[title_idx][:top*2]
  
  #검색 영화 제외
  if title_idx in movie_list:
    movie_list = movie_list[movie_list != title_idx]
  print(movie_list)
  print(3337 in movie_list)
  return dataframe.iloc[movie_list].sort_values('weighted_vote', ascending=False)[:top]

#가중치 점수를 이용한 top10영화 추천
movie_top10 = find_movie(df_movies, overview_similarity_sorted, 'superman')
movie_top10

[ 870   10 2433 ... 1147 3197    0]
True


,title,budget,genres,keywords,overview,popularity,release_date,runtime,vote_average,vote_count,weighted_vote
1881,The Shawshank Redemption,25000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 80, ""name...","[{""id"": 378, ""name"": ""prison""}, {""id"": 417, ""n...","framed 1940s double murder his wife lover, ups...",136.747729,1994-09-23,142.0,8.5,8205,8.395924
3337,The Godfather,6000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 80, ""name...","[{""id"": 131, ""name"": ""italy""}, {""id"": 699, ""na...","spanning years 1945 1955, chronicle fictional ...",143.659698,1972-03-14,175.0,8.4,5893,8.263433
662,Fight Club,63000000,"[{""id"": 18, ""name"": ""Drama""}]","[{""id"": 825, ""name"": ""support group""}, {""id"": ...",a ticking-time-bomb insomniac slippery soap sa...,146.757391,1999-10-15,139.0,8.3,9413,8.216358
3232,Pulp Fiction,8000000,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 80, ""n...","[{""id"": 396, ""name"": ""transporter""}, {""id"": 14...","a burger-loving hit man, his philosophical par...",121.463076,1994-10-08,154.0,8.3,8428,8.206994
65,The Dark Knight,185000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 28, ""name...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",batman raises stakes his war crime. help lt. j...,187.322927,2008-07-16,152.0,8.2,12002,8.136858
1818,Schindler's List,22000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 36, ""name...","[{""id"": 1382, ""name"": ""factory""}, {""id"": 1631,...",the true story how businessman oskar schindler...,104.469351,1993-11-29,195.0,8.3,4329,8.125882
3865,Whiplash,3300000,"[{""id"": 18, ""name"": ""Drama""}]","[{""id"": 1416, ""name"": ""jazz""}, {""id"": 1523, ""n...","under direction ruthless instructor, talented ...",192.528841,2014-10-10,105.0,8.3,4254,8.123058
809,Forrest Gump,55000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""id"": 422, ""name"": ""vietnam veteran""}, {""id""...",a man low iq accomplished great things his lif...,138.133331,1994-07-06,142.0,8.2,7927,8.105850
2294,Spirited Away,15000000,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 12, ""na...","[{""id"": 616, ""name"": ""witch""}, {""id"": 970, ""na...",a ten year old girl wanders away from parents ...,118.968562,2001-07-20,125.0,8.3,3840,8.105662
2731,The Godfather: Part II,13000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 80, ""name...","[{""id"": 700, ""name"": ""italo-american""}, {""id"":...","in continuing saga corleone crime family, youn...",105.792936,1974-12-20,200.0,8.3,3338,8.079359


In [ ]:
word_split = df_movies['overview'].iloc[0].split(' ')
test_movie = df_movies['overview'].iloc[634]
print(test_movie)

for word in word_split:
  word_space = ' {} '.format(word)
  test_moive = test_movie.replace(word_space, '  ~~~~~~~~~~~~    ')
test_movie

set 22nd century, matrix tells story computer hacker joins group underground insurgents fighting vast powerful computers now rule earth.


'set 22nd century, matrix tells story computer hacker joins group underground insurgents fighting vast powerful computers now rule earth.'